In [5]:
%pip install pydantic-output-parser

DEPRECATION: Loading egg at /home/nati/miniconda3/envs/rag/lib/python3.12/site-packages/huggingface_hub-0.20.3-py3.8.egg is deprecated. pip 24.3 will enforce this behaviour change. A possible replacement is to use pip for package installation.. Discussion can be found at https://github.com/pypa/pip/issues/12330
ERROR: Could not find a version that satisfies the requirement pydantic-output-parser (from versions: none)
ERROR: No matching distribution found for pydantic-output-parser
Note: you may need to restart the kernel to use updated packages.


In [3]:
%pip install langchain_openai

DEPRECATION: Loading egg at /home/nati/miniconda3/envs/rag/lib/python3.12/site-packages/huggingface_hub-0.20.3-py3.8.egg is deprecated. pip 24.3 will enforce this behaviour change. A possible replacement is to use pip for package installation.. Discussion can be found at https://github.com/pypa/pip/issues/12330
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 246.4/246.4 kB 784.7 kB/s eta 0:00:0000:0100:01
  Attempting uninstall: langchain-core
    Found existing installation: langchain-core 0.1.24
    Uninstalling langchain-core-0.1.24:
      Successfully uninstalled langchain-core-0.1.24
Note: you may need to restart the kernel to use updated packages.


In [2]:
%pip install elasticsearch python-dotenv

DEPRECATION: Loading egg at /home/nati/miniconda3/envs/rag/lib/python3.12/site-packages/huggingface_hub-0.20.3-py3.8.egg is deprecated. pip 24.3 will enforce this behaviour change. A possible replacement is to use pip for package installation.. Discussion can be found at https://github.com/pypa/pip/issues/12330
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 432.1/432.1 kB 1.2 MB/s eta 0:00:0000:0100:01m
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.9/59.9 kB 1.6 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [1]:
from dotenv import load_dotenv
import os
from langchain_community.vectorstores.elasticsearch import ElasticsearchStore
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.memory import ConversationBufferMemory
from langchain.chains import ConversationalRetrievalChain
import PyPDF2
from langchain.chat_models import ChatOpenAI
from elasticsearch import Elasticsearch
from langchain.prompts import PromptTemplate
from sentence_transformers import SentenceTransformer, util

In [6]:
# load_dotenv()
es_model_id = '.elser_model_2_linux-x86_64'
index_name = "elser_index_vb_test_2"
llm_model_id = "meta-llama/llama-2-13b-chat"
wx_url = "https://us-south.ml.cloud.ibm.com"
wx_project_id = 'cc83aaaf-02bb-4d97-9a20-4ab4ce02cee6'

In [24]:
pdf_docs=["./Raptor Contract.pdf"]

In [8]:
def prepare_docs(pdf_docs):
    docs = []
    metadata = []
    content = []

    for pdf in pdf_docs:

        pdf_reader = PyPDF2.PdfReader(pdf)
        for index, text in enumerate(pdf_reader.pages):
            doc_page = {'title': pdf + " page " + str(index + 1),
                        'content': pdf_reader.pages[index].extract_text()}
            docs.append(doc_page)
    for doc in docs:
        content.append(doc["content"])
        metadata.append({
            "title": doc["title"]
        })
    print("Content and metadata are extracted from the documents")
    return content, metadata

In [9]:
def get_text_chunks(content, metadata):
    text_splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(
        chunk_size=512,
        chunk_overlap=256,
    )
    split_docs = text_splitter.create_documents(content, metadatas=metadata)
    print(f"Documents are split into {len(split_docs)} passages")
    return split_docs

In [22]:
def ingest_and_get_vector_store(split_docs):
    vector_store = ElasticsearchStore(
                    es_cloud_id= "8d770dd7f5e74f5da7014ec9069249b8:dXMtY2VudHJhbDEuZ2NwLmNsb3VkLmVzLmlvOjQ0MyRhYjI5YzA5NmVlZmQ0OTgzOWM0YTdiYzdjMzVkOWFiYiRiNGM2ZDU4ZjA0MmM0YTEzYmRiMTgzMjhjY2NmZThmZA==",
                    es_api_key="c0p0VTFvMEJHY3FJNUtFVE5ST2I6SmxPZnNjRkRUWUt0SmRTV05senlqdw==",
                    index_name=index_name,
                    strategy=ElasticsearchStore.SparseVectorRetrievalStrategy(model_id=es_model_id)
                    )
    documents = vector_store.from_documents(
        split_docs,
        es_cloud_id= "8d770dd7f5e74f5da7014ec9069249b8:dXMtY2VudHJhbDEuZ2NwLmNsb3VkLmVzLmlvOjQ0MyRhYjI5YzA5NmVlZmQ0OTgzOWM0YTdiYzdjMzVkOWFiYiRiNGM2ZDU4ZjA0MmM0YTEzYmRiMTgzMjhjY2NmZThmZA==",
        es_api_key="c0p0VTFvMEJHY3FJNUtFVE5ST2I6SmxPZnNjRkRUWUt0SmRTV05senlqdw==",
        index_name=index_name,
        strategy=ElasticsearchStore.SparseVectorRetrievalStrategy(model_id=es_model_id)
    )
    print("Documents indexed and vector Store returned")

    return vector_store


In [11]:
template ="""[INST]You are a helpful, respectful, and honest assistant. 
Always answer as helpfully as possible, while being safe. Be brief in your answers. Your answers should not include any harmful, unethical, racist, sexist, toxic, dangerous, or illegal content. Please ensure that your responses are socially unbiased and positive in nature.If you don\\'\''t know the answer to a question, please do not share false information. \n Answer with no more than 150 words, in 2 or 3 sentences. If you cannot base your answer on the given document, please state that you do not have an answer.\n\n{question} Answer with no more than 200 words. If you cannot base your answer on the given document, please state that you do not have an answer. do not include a question in your response. dont prompt to make select correct answers[/INST]"""

template = """[INST]
As an AI, provide accurate and relevant information based on the provided document. Your responses should adhere to the following guidelines:
- Answer the question based on the provided documents.
- Be direct and factual, limited to 50 words and 2-3 sentences. Begin your response without using introductory phrases like yes, no etc.
- Maintain an ethical and unbiased tone, avoiding harmful or offensive content.
- If the document does not contain relevant information, state "I cannot provide an answer based on the provided document."
- Avoid using confirmatory phrases like "Yes, you are correct" or any similar validation in your responses.
- Do not fabricate information or include questions in your responses.
- do not prompt to select answers. do not ask me questions

{question}


[/INST]
"""

#template = """Given the document and the current conversation between a user and an agent, your task is as follows: Answer any user query by using information from the document. The response should be detailed."""



def get_conversation_chain(vector_store):
    openai_llm = ChatOpenAI(model_name="gpt-3.5-turbo")
    retriever = vector_store.as_retriever()
    CONDENSE_QUESTION_PROMPT = PromptTemplate.from_template(template)

    memory = ConversationBufferMemory(
        memory_key='chat_history', return_messages=True, output_key='answer')

    conversation_chain = (ConversationalRetrievalChain.from_llm
                          (llm=openai_llm,
                           retriever=retriever,
                           condense_question_prompt=CONDENSE_QUESTION_PROMPT,
                           memory=memory,
                           return_source_documents=True))
    print("Conversational Chain created for the LLM using the vector store")
    return conversation_chain

In [12]:
def validate_answer_against_sources(response_answer, source_documents):
    model = SentenceTransformer('all-MiniLM-L6-v2')
    similarity_threshold = 0.5  
    source_texts = [doc.page_content for doc in source_documents]

    answer_embedding = model.encode(response_answer, convert_to_tensor=True)
    source_embeddings = model.encode(source_texts, convert_to_tensor=True)

    cosine_scores = util.pytorch_cos_sim(answer_embedding, source_embeddings)


    if any(score.item() > similarity_threshold for score in cosine_scores[0]):
        return True  

    return False  

In [25]:
# Now that we have crafted all the necessary functions, it's time to put them into action and test their functionality.
content, metadata = prepare_docs(pdf_docs)
# Content and metadata are extracted from the documents
split_docs = get_text_chunks(content, metadata)
# Documents are split into 3 passages
vectorstore = ingest_and_get_vector_store(split_docs)
# Documents indexed and vector Store returned
conversation_chain=get_conversation_chain(vectorstore)

Content and metadata are extracted from the documents
Documents are split into 333 passages


Error adding texts: 333 document(s) failed to index.
First error reason: Could not find trained model [.elser_model_2_linux-x86_64]


BulkIndexError: 333 document(s) failed to index.